<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png" width="40px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Chat Completion for Decisions</span>
</div>

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

----
### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [6]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.9"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.9"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.9 Microsoft.Extensions.Configuration.Json, 9.0.9

In [7]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.9"
#r "nuget: Microsoft.SemanticKernel, 1.65.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.OpenAI;


Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.9 Microsoft.SemanticKernel, 1.65.0

Using Azure OpenAI Service


### Step 2 - Execute Decision Chat Completion with Prompt Execution Settings 

> 📜 **_"If you can’t describe what you are doing as a process, you don’t know what you’re doing."_** 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [8]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with 
// LLMs are trained on a diverse range of data and can provide insights on a wide range of topics like decision frameworks
// SLMS (smaller LLMs) are trained on a more specific range of data and may not provide insights on all topics
var simpleDecisionPrompt = """
Provide a list of 5 decision frameworks that can help improve the quality of decisions.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
""";

PromptExecutionSettings promptExecutionSettings = new();

if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 1500,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}

// Execute the chat completion request using the prompt and settings defined above
var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, promptExecutionSettings);

// Display the result of the chat completion request as Markdown
chatResult.Content.DisplayAs("text/markdown");

### Five decision frameworks to improve decision quality

Below are five practical frameworks, each with a short description, when to use it, core steps, and quick pros/cons to help you pick the right one for a situation.


#### 1) Weighted Decision Matrix (a.k.a. Multi-criteria Decision Analysis)

What it is: A structured way to compare options by scoring them against weighted criteria.

When to use: When you must choose between multiple alternatives with several different, measurable criteria.

Core steps:
- List options and the decision criteria.
- Assign weights to criteria by importance.
- Score each option on each criterion (e.g., 1–10).
- Multiply scores by weights and sum to get weighted totals.
- Compare totals and perform sensitivity checks.

Pros:
- Makes trade-offs explicit and repeatable.
- Good for group decisions and prioritization.

Limitations:
- Scores and weights can be subjective; require calibration.


  

#### 2) Cost–Benefit Analysis with Expected Value / Probability

What it is: Quantifies benefits and costs and uses probabilities to handle uncertainty (expected value).

When to use: For choices with measurable costs/benefits and uncertain outcomes, especially investments or policy decisions.

Core steps:
- Enumerate possible outcomes for each option.
- Estimate monetary (or utility) costs and benefits for each outcome.
- Assign probabilities to outcomes.
- Calculate expected value (sum of value × probability).
- Compare expected values and consider decision rules (max EV, risk tolerance).

Pros:
- Brings uncertainty and scale into the comparison.
- Useful for rational, repeatable decisions.

Limitations:
- Requires reasonable probability and value estimates; sensitive to these inputs.


  

#### 3) Pre-mortem

What it is: A prospective analysis where team members imagine a future failure and generate reasons why it happened.

When to use: Before launching a project or major change to uncover hidden risks and fixable flaws.

Core steps:
- Assemble the decision team before implementation.
- Assume the project failed spectacularly; ask “Why?”
- Collect many failure causes from participants independently.
- Prioritize and create mitigation actions for the most plausible causes.
- Reassess decision with mitigations in place.

Pros:
- Surface issues that optimism and groupthink hide.
- Quick, low-cost way to improve resilience.

Limitations:
- Not a substitute for quantitative analysis; depends on candid participation.


  

#### 4) OODA Loop (Observe, Orient, Decide, Act)

What it is: A rapid decision cycle for situations that require speed and adaptability.

When to use: In fast-changing environments (operations, crisis response, competitive moves) where feedback loops matter.

Core steps:
- Observe: gather current data and changes.
- Orient: interpret observations, update mental models.
- Decide: pick the best short-term action.
- Act: execute and then immediately observe results to iterate.

Pros:
- Encourages agility and continuous learning.
- Helps outpace competitors or adapt to disruptions.

Limitations:
- Less suited to highly complex decisions requiring deep analysis; can encourage hasty choices if misapplied.


  

#### 5) Eisenhower Matrix (Urgent vs. Important)

What it is: A prioritization grid that separates tasks into four quadrants by urgency and importance.

When to use: For personal and team task prioritization and time-management decisions.

Core steps:
- Classify tasks into: (A) Important & Urgent, (B) Important & Not Urgent, (C) Not Important & Urgent, (D) Not Important & Not Urgent.
- Do A immediately; schedule B; delegate C; eliminate D.
- Revisit regularly and reclassify as context changes.

Pros:
- Simple and fast way to focus on high-impact activities.
- Reduces firefighting and supports strategic work.

Limitations:
- Doesn’t address complex trade-offs across projects; subjective classification.


  

If you want, I can: apply one of these frameworks to a decision you’re facing, provide a template (e.g., a weighted matrix spreadsheet), or suggest when to combine two frameworks for better results. Which would you like?

----
### Step 3 - Execute Decision Chat Completion with Streaming 

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and API services. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [10]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt, promptExecutionSettings))
{
    Console.Write(content);
}

### Five decision frameworks to improve decision quality


#### 1. Weighted scoring model (Multi‑criteria decision analysis)
- What it is: List options, define decision criteria, assign weights to criteria by importance, score each option against criteria, compute weighted totals.
- Key steps: choose criteria → assign weights (sum = 1) → score options (e.g., 0–10) → multiply & sum → rank.
- When to use: comparing multiple alternatives with conflicting objectives (e.g., vendors, product features).
- Strengths: transparent, repeatable, helps quantify tradeoffs and document rationale.
- Limitations: sensitive to chosen weights/scores; still relies on judgment.

  

#### 2. Decision trees and expected value
- What it is: A graphical model mapping choices, uncertain outcomes, probabilities, and payoffs; compute expected values to choose the best option.
- Key steps: draw tree → estimate branches and probabilities → estimate payoffs/costs → compute expected value → consider risk and sensitiv

----
### Step 4 - Execute Decision Chat Completion with a System Prompt & Chat History 

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt. These are usually global instructions that set the "overall rules" for interacting with the LLMs.
* User Message - A message from the user
* Assistant Message - A message from the LLM. This is a message generated from an assistant or an agent. 

Identifying the messages from which role (user) it came from can help the LLM improve its own reasoning responses. This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [11]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

// Simple instruction prompt to list 5 (five) decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five Decision Frameworks that can help improve the quality of decisions.
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

// Note: The chat completion service can also be used with a chat history object, which allows for a more conversational context.
var chatResponse = string.Empty;

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add the assistant's response to the chat history
chatHistory.AddAssistantMessage(chatResponse);

| Framework | When to use | Core steps | Inputs needed | Strengths | Limitations |
|---|---:|---|---|---|---|
| Weighted Scoring / Multi‑Criteria Decision Analysis (MCDA) | Choosing between multiple options that must be compared across several qualitative and quantitative criteria (e.g., vendor selection, project prioritization) | 1) List options and decision criteria 2) Assign weights to criteria (importance) 3) Score each option on each criterion 4) Calculate weighted sums and rank options 5) Run sensitivity checks on weights | Options, relevant criteria, relative weights, scores (from experts/data) | Transparent, easy to communicate, mixes qualitative and quantitative factors, supports sensitivity analysis | Subjective weights and scores can bias results; poor at representing uncertainty or interdependent criteria |
| Decision Tree with Expected Value (EV) | Sequential or staged decisions with probabilistic outcomes and clear payoff/costs (e.g., go/no‑go investment, product launch stages) | 1) Draw decision and chance nodes 2) Assign probabilities and payoffs/costs to outcomes 3) Roll back the tree to compute EVs and optimal branches 4) Perform sensitivity analysis on probabilities/payoffs | Possible actions, outcome states, probabilities, payoffs/costs for each branch | Clarifies sequential options, makes assumptions explicit, supports value of information and option‑value analysis | Requires credible probabilities and payoffs; trees grow complex for many stages; may hide model uncertainty |
| Probabilistic Simulation (Monte Carlo) | Decisions where inputs are uncertain and you care about distributions, tail risks, and probabilities of meeting targets (e.g., portfolio risk, project cost/time risk) | 1) Build model linking inputs to outcomes 2) Define probability distributions and correlations for uncertain inputs 3) Run many simulations 4) Analyze output distributions, percentiles, and risk metrics | Model structure, probability distributions for uncertain inputs, correlation assumptions, computational tool | Captures uncertainty and variability, shows full outcome distribution and tail risk, supports scenario and sensitivity analysis | Requires good distributional estimates and correlation assumptions; can produce false precision if inputs are poor; needs computational effort |
| Pre‑mortem / Red Teaming (Prospective Hindsight) | High‑impact or high‑uncertainty decisions where you need to surface hidden failure modes and reduce overconfidence (e.g., strategic initiatives, major launches) | 1) Assume the decision failed spectacularly 2) Ask teams to list reasons why it failed 3) Aggregate and prioritize failure modes 4) Implement mitigations or rethink the plan | Draft plan/decision, diverse stakeholders, time for structured session | Low cost, reveals blind spots and cognitive biases, builds shared situational awareness and actionable mitigations | Relies on candid participation; qualitative (not quantitative); can generate many items that need prioritization |
| PDCA (Plan–Do–Check–Act) / Iterative Learning | Decisions under uncertainty where experimentation and continuous improvement reduce risk (e.g., product development, process changes) | 1) Plan: define hypothesis, success metrics, and small experiment 2) Do: run the experiment/implement pilot 3) Check: measure results against metrics 4) Act: adopt, adapt, or abandon and iterate | Clear hypothesis, measurable metrics, ability to run small tests/experiments, data collection | Encourages small steps, rapid learning, reduces downside risk, adapts to new information | Slower to reach large scale, requires discipline to measure and iterate, may be resisted in non‑experimental cultures |

In the next step, an additional prompt instruction will be added to the chat history. From the 5 decision frameworks provided in the chat history, Generative AI is asked to recommed a framework best suited for the militaty intelligence community. Notice that the previous chat history is automatically provided to provide additional intelligence context. 

> 📜 **_"Take time to deliberate, but when the time for action comes, stop thinking and go in."_**
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [12]:
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupQuestionPartTwo = """
Which of the 5 decision frameworks listed above is best suited for the military intelligence community?  
Think carefully step by step about what decision frameworks are needed to answer the query.  
Select only the single best framework. 
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupQuestionPartTwo);

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

| Step | Key points considered | Evaluation / Evidence | Result |
|---|---:|---|---|
| 1) Characterize military intelligence decision needs | High uncertainty, adversary deception, high consequences, need for early warning and alternative hypotheses, frequent sparse/ambiguous data, severe cost of surprise | These traits favor methods that surface hidden assumptions, model adversary intent, and work with limited quantitative data | Establish baseline requirements for suitable framework |
| 2) Enumerate core capabilities required | 1) Expose cognitive biases and groupthink 2) Generate adversary‑centric scenarios 3) Produce actionable indicators/mitigations 4) Work effectively with qualitative or limited data 5) Be time‑efficient and repeatable | Capability match will determine which framework aligns best with IC operational constraints | Use these capabilities as evaluation criteria |
| 3) Quick mapping of the five frameworks to capabilities | MCDA: good for resource/prioritization but needs clear criteria and weights. Decision Tree (EV): good for sequential choices with reliable probabilities. Monte Carlo: excellent for quantifying uncertainty when distributions exist. PDCA: good for iterative learning and experiments. Pre‑mortem/Red Teaming: specifically targets biases, deception, alternative hypotheses, and indicator development with low data needs. | Only Pre‑mortem/Red Teaming directly addresses adversary thinking, deception, and bias mitigation in low‑data, high‑consequence environments typical of military intelligence | Narrow candidates against IC needs |
| 4) Trade‑offs and practicality | Quantitative methods (Monte Carlo, Decision Trees) add rigor when data/probabilities are available but can produce false confidence with poor inputs. MCDA helps prioritization but doesn't unmask deception. PDCA requires ability to run experiments/pilots. Pre‑mortem/Red Teaming is low‑cost, rapidly surfaces blind spots and adversary strategies but is qualitative and depends on candid participation. | Given frequent data scarcity and the paramount need to avoid surprise, a framework that reveals failure modes and adversary options is often most impactful operationally | Pre‑mortem/Red Teaming offers the strongest fit for core IC needs |
| 5) Single best framework selection | Which single framework best meets IC priorities (bias mitigation, adversary analysis, indicator generation, low‑data operation) | Pre‑mortem / Red Teaming (Prospective Hindsight) — best suited as the primary single framework for the military intelligence community | Selected: Pre‑mortem / Red Teaming |
| 6) Caveats and recommended complements | Limitation: qualitative, requires diverse/candid participation, can produce many unprioritized items. Recommendation: pair Red Teaming with probabilistic methods (Monte Carlo or Decision Trees) and MCDA for resource allocation when reliable data exists; use PDCA for iterative collection/validation of indicators. | Use Red Teaming as the central practice but integrate quantitative frameworks where data quality supports them to improve rigor and decision traceability | Balanced approach: Red Teaming primary + targeted quantitative complements |

### Step 5 - Execute Decision Chat Completion with a Multi-Turn Conversation Scenario 

> 📜 **_"Price is what you pay. Value is what you get."_**  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

Semantic Kernel can maintain state with the ChatHistory object. This allows natural converation between the user and AI to be persisted for future use. It can be saved for future reference, it can be replayed, it can be forked in certain spots to try different scenarios, it can be used as a starting point for other conversations etc. 

In the scenario below, let's build a multi-turn conversation about a decision over a purchase for a fitness tracker. Notice that each turn the appropriate Chat History roles are populated with the appropriate prompts and assistant (AI) responses. This in effect acts as the Gathering Intelligence step of the Decision Intelligence framework as the conversation information has been persisted and can be re-used as context for the decision scenario.

In [13]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPromptDecisionConversation = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// User prompt to start a decision conversation about buying a fitness tracker
var userPromptDecisionConversationTurn1 = """
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 
""";

var assistantResponseDecisionConversationTurn1 = """
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.
""";

var userPromptDecisionConversationTurn2 = """
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?
""";

var assistantResponseDecisionConversationTurn2 = """
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.
""";

var userPromptDecisionConversationTurn3 = """
Are there any popular models you would recommend for that balance of battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn3 = """
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.
""";

var userPromptDecisionConversationTurn4 = """
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn4 = """
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 
""";

// Create a new chat history with a multi-turn conversation about buying a fitness tracker
// Notice how the conversation flow uses different roles (user and assistant) to simulate a realistic decision-making dialogue
var chatHistoryDecisionConversation = new ChatHistory();
chatHistoryDecisionConversation.AddSystemMessage(systemPromptDecisionConversation);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn1);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn1);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn2);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn2);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn3);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn3);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn4);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn4);


Using the above conversation history as the "Gathered Intelligence" betweeen the user and the AI assistant, let's make a final decision and ask for final feedback and a decision evaluation. This "Gathered Intelligence" for the decision can be persisted, re-loaded, simulated multiple times, applied to different AI systems to optimize decisions further.  

In [14]:
var userPromptDecisionConversationFinalDecision = """
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 

Any quick final thoughts or insights on my decision? 
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationFinalDecision);

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryDecisionConversation))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add Assistant message to the chat history
chatHistoryDecisionConversation.AddAssistantMessage(chatResponse);

Nice choice — the Inspire 3 is a solid, budget-friendly pick if battery life and sleep tracking are priorities. Quick, practical final thoughts to get the most out of it:

Key strengths
- Multi-day battery life: expect around a week to 10 days with typical use (turning on power-hungry features will shorten that).
- Good basic sleep tracking: automatic sleep detection, sleep stages (light/deep/REM) and a sleep score through the Fitbit app.
- Useful sensors for the price: continuous heart rate and SpO2 estimates that help refine sleep insights.

What to expect (realistically)
- Sleep data is useful for trends and spotting big changes but isn’t a medical-grade diagnosis.
- Accuracy improves when the band fits snugly but comfortably — not too loose and not squeezing your wrist.
- You’ll need to charge occasionally (usually weekly); a short daily top-up isn’t necessary unless you prefer it.

Quick setup & usage tips
- Wear it a bit higher on your wrist (about 1–2 finger widths above the bone) for better heart-rate and SpO2 readings.
- Let it sync with the Fitbit app overnight the first few nights so the app can learn your patterns.
- Keep firmware updated — Fitbit pushes improvements to tracking and battery behavior via updates.

Battery-saving hints
- Limit continuous notifications and raise-to-wake brightness if you want longer runtime.
- Turn off features you don’t use (e.g., bg heart-rate sampling frequency is often adjustable).
- Use a short weekly charging routine (e.g., charge while showering on a chosen day) to keep tracking continuous.

App, privacy & subscription
- The Fitbit app gives most insights for free; Fitbit Premium (paid) adds deeper analysis and guided programs if you want them.
- Review Fitbit’s privacy policy if you’re concerned about how health data is handled or shared.

Accessories & practicalities
- Check for compatible replacement bands you like for comfort and style.
- Note it uses a proprietary charger — keep a spare if you travel.

When to reconsider
- If you later want advanced athlete metrics (VO2max, advanced run dynamics) or native apps, consider upgrading to Charge/Garmin/Apple options.
- If you need medical-grade sleep apnea detection, consult a clinician — trackers can flag issues but not replace diagnosis.

If you want, I can:
- Walk through an ideal initial setup checklist.
- Suggest specific bands or settings to maximize battery and sleep accuracy.
- Compare Inspire 3 vs. Charge 5 quickly if you’re curious about small upgrades.

Would you like a short setup checklist now?

----
### Step 6 - Inspect & Optimize Gathered Intelligence of Chat History 

The Semantic Kernel Chat History is a transparent construct that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

> **📝 Note:** In the Decision Intelligence framework, chat history can serve as a form of gathered intelligence. Interactions among users, AI models, processes, and agents provide valuable context for effective decision-making. It’s highly recommended to persist chat history objects (agent threads), especially during decision optimization, to ensure critical information is retained and accessible.  

In [15]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistoryDecisionConversation.Count());

// Change this to a string builder and show as markdown
var stringBuilderChatHistory = new StringBuilder();
foreach (var message in chatHistoryDecisionConversation)
{
    // add a new line for each message
    stringBuilderChatHistory.AppendLine($"**{message.Role.ToString().ToUpper()}**:");
    stringBuilderChatHistory.Append($"{message.Content.Replace("#", string.Empty)}");
    stringBuilderChatHistory.AppendLine("\n");
}

// Display the chat history as Markdown
stringBuilderChatHistory.ToString().DisplayAs("text/markdown");

Number of chat interactions: 11


**SYSTEM**:
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

**USER**:
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 

**ASSISTANT**:
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.

**USER**:
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?

**ASSISTANT**:
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.

**USER**:
Are there any popular models you would recommend for that balance of battery life and sleep tracking?

**ASSISTANT**:
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.

**USER**:
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?

**ASSISTANT**:
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 

**USER**:
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 

Any quick final thoughts or insights on my decision? 

**ASSISTANT**:
Nice choice — the Inspire 3 is a solid, budget-friendly pick if battery life and sleep tracking are priorities. Quick, practical final thoughts to get the most out of it:

Key strengths
- Multi-day battery life: expect around a week to 10 days with typical use (turning on power-hungry features will shorten that).
- Good basic sleep tracking: automatic sleep detection, sleep stages (light/deep/REM) and a sleep score through the Fitbit app.
- Useful sensors for the price: continuous heart rate and SpO2 estimates that help refine sleep insights.

What to expect (realistically)
- Sleep data is useful for trends and spotting big changes but isn’t a medical-grade diagnosis.
- Accuracy improves when the band fits snugly but comfortably — not too loose and not squeezing your wrist.
- You’ll need to charge occasionally (usually weekly); a short daily top-up isn’t necessary unless you prefer it.

Quick setup & usage tips
- Wear it a bit higher on your wrist (about 1–2 finger widths above the bone) for better heart-rate and SpO2 readings.
- Let it sync with the Fitbit app overnight the first few nights so the app can learn your patterns.
- Keep firmware updated — Fitbit pushes improvements to tracking and battery behavior via updates.

Battery-saving hints
- Limit continuous notifications and raise-to-wake brightness if you want longer runtime.
- Turn off features you don’t use (e.g., bg heart-rate sampling frequency is often adjustable).
- Use a short weekly charging routine (e.g., charge while showering on a chosen day) to keep tracking continuous.

App, privacy & subscription
- The Fitbit app gives most insights for free; Fitbit Premium (paid) adds deeper analysis and guided programs if you want them.
- Review Fitbit’s privacy policy if you’re concerned about how health data is handled or shared.

Accessories & practicalities
- Check for compatible replacement bands you like for comfort and style.
- Note it uses a proprietary charger — keep a spare if you travel.

When to reconsider
- If you later want advanced athlete metrics (VO2max, advanced run dynamics) or native apps, consider upgrading to Charge/Garmin/Apple options.
- If you need medical-grade sleep apnea detection, consult a clinician — trackers can flag issues but not replace diagnosis.

If you want, I can:
- Walk through an ideal initial setup checklist.
- Suggest specific bands or settings to maximize battery and sleep accuracy.
- Compare Inspire 3 vs. Charge 5 quickly if you’re curious about small upgrades.

Would you like a short setup checklist now?



By persisting the Decision Intelligence chats and making them available for inspection, can further help the decision-making process. For example you can load the Chat History summarize it or even have AI recommend optimizations for it based on the conversation. 

> **📝 Note:** The process below is simplified to illustrate the power of Gathering Intelligence with user <--> AI interactions. More advanced best practices will be introduced in further sections. 

In [16]:
var userPromptSummarizeTheDecisionChat = """
Briefly summarize the chat conversation about buying a fitness tracker.
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptSummarizeTheDecisionChat);

var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
chatResponse = chatContent.Content;
chatResponse.DisplayAs("text/markdown");

// Optimize the chat history for any future decision interactions
var userPromptOptimizeTheDecisionChat = """
Based on the chat conversation so far...
Provide some recommendations on how the decision interactions could be optimized for decision-making.
What are some considerations for the AI to help the use make even better decisions?
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptOptimizeTheDecisionChat);

var chatContentOptimization = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
var chatResponseOptimization = chatContentOptimization.Content;
chatResponseOptimization.DisplayAs("text/markdown");

- You asked which fitness-tracker features matter; I listed key considerations (fitness/health features, battery, comfort, compatibility, price).
- You prioritized battery life and sleep tracking; I explained typical battery ranges (about 2–3 days for feature-rich watches up to ~10–14 days for minimalist trackers) and noted modern trackers can detect sleep stages and give useful trends but aren’t medical-grade.
- I recommended models balancing sleep tracking and battery: Fitbit Inspire 3 and Charge 5, Garmin Vivosmart 5, and Xiaomi Mi Band 7.
- We compared the Apple Watch: strong features and health monitoring but much shorter battery (~18–24 hours), so daily charging can disrupt continuous sleep tracking.
- You chose the Fitbit Inspire 3. I gave practical tips: expect ~1 week to 10 days battery, wear snugly for better readings, sync and update the app/firmware, use a weekly charging routine, and remember sleep data is for trends not diagnosis.
- I also noted Fitbit app/Premium and privacy considerations, replacement bands, and when to consider upgrading (advanced athletic metrics or medical needs).

Summary of the conversation (brief)
- You asked which fitness-tracker features matter; I listed common considerations (fitness/health features, battery life, comfort, compatibility, price).
- You said battery life and sleep tracking are most important; I described typical battery ranges (2–3 days up to 10–14 days) and how modern trackers handle sleep stages and SpO2.
- I recommended models that balance battery and sleep tracking (Fitbit Inspire 3, Charge 5, Garmin Vivosmart 5, Xiaomi Mi Band 7).
- You asked about the Apple Watch; I explained it has strong features but shorter battery life (~18–24 hr), so daily charging may interfere with continuous sleep tracking.
- You decided on the Fitbit Inspire 3 and I gave practical setup, accuracy, battery-saving, and privacy tips, plus offered to provide a setup checklist or comparisons.

How to optimize decision interactions (recommendations)
1. Start by clarifying and prioritizing goals
   - Ask the short list: must-haves, nice-to-haves, deal-breakers (e.g., battery ≥7 days, automatic REM detection, budget ≤ $150).
2. Elicit usage context and constraints
   - Capture daily routine (sleep schedule, charging habits), primary activities (running, weightlifting), phone OS, wrist size, and aesthetic preferences.
3. Use a structured trade-off framework
   - Present a compact multi-criteria comparison (rank or score options across battery, sleep accuracy, sensors, price, app, comfort).
4. Narrow options with progressive filtering
   - Start broad, then filter by constraints (e.g., battery >7 days → show 3–4 best matches) to avoid overload.
5. Make trade-offs explicit and quantitative
   - Translate features into expected outcomes (e.g., “Expect ~7–10 days battery with moderate use; enabling continuous SpO2 reduces this by ~20%”).
6. Recommend small experiments before committing
   - Suggest trialing devices (return policies), wearing a temporary band to test comfort, or comparing sleep data side-by-side for a few nights.
7. Provide decision checkpoints and action steps
   - Offer a short checklist for purchase, setup, and a 2-week evaluation plan to decide whether to keep or return.

Considerations for the AI to help make better decisions
- Personalization: tailor recommendations by learning preferences (budget, phone OS, activity types) and behavior (charging habits, tolerance for false positives).
- Preference elicitation: use simple, guided questions or sliders to quantify trade-offs and produce ranked options automatically.
- Transparency & uncertainty: state confidence levels, typical variability (accuracy ranges), and cite sources or manufacturer specs where relevant.
- Comparative scoring & sensitivity analysis: show how recommendations change if the user adjusts a priority (e.g., increase importance of sleep accuracy from medium to high).
- Scenario simulation: outline likely daily routines (charging schedule, sleep-tracking continuity) for each device to show practical impacts.
- Evidence & limitations: flag that trackers are not medical devices, summarize common accuracy limitations (wrist placement, movement, skin tone, arrhythmias), and suggest when to seek clinical evaluation.
- Actionable next steps: give an explicit small plan (purchase + 2-week trial + data checks + return deadline) and what metrics to watch (sleep score trends, battery drain).
- Privacy & data handling: remind users to review vendor privacy policies and discuss what data syncs to cloud/third parties.
- User control & explainability: offer plain-language explanations of how metrics are derived (HR variability → sleep stage inference) and how to interpret them.
- Practical supports: propose band-fit tips, charging routines, third-party apps or integrations, and accessories to improve experience.

If you want, I can:
- Create a short prioritized checklist (must-haves vs nice-to-haves) for you to confirm.
- Build a simple scoring table comparing 3–4 specific models (Inspire 3, Charge 5, Vivosmart 5, Mi Band 7) against your priorities.